In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [13]:
# parameter setting
batch_size = 32
learning_rate = 0.001
epoch = 15

# GPU setting
cuda_available = torch.cuda.is_available()
device = torch.device('cuda' if cuda_available else 'cpu')

print('Current cuda device is', device)
!nvidia-smi

Current cuda device is cuda
Thu Apr 25 19:06:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0              32W /  70W |    291MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+---------------------------------------

In [14]:
# import dataset
train_data = datasets.MNIST(root = './data/train/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_data = datasets.MNIST(root = './data/test/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=True)

In [15]:
# Deep Learning Model setting
class Convolution_Neural_Networks(nn.Module):
    def __init__(self):
        super(Convolution_Neural_Networks, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1, padding = 'same')
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 'same')
        self.dropout = nn.Dropout2d(0.25)
        self.relu = nn.ReLU()
        self.max_pooling = nn.MaxPool2d(kernel_size = 2)
        self.fc1 = nn.Linear(3136, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pooling(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pooling(x)

        x = self.dropout(x)
        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [16]:
# apply GPU to model
model = Convolution_Neural_Networks().cuda(device)
# apply CPU to model
#model = Convolution_Neural_Networks().cpu

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()


In [17]:
#Train
i=1
for epoch in range(epoch):
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print("Train Step: {}\tLoss : {:3f}".format(i, loss.item()))
        i += 1

Train Step: 1000	Loss : 0.047688
Train Step: 2000	Loss : 0.056366
Train Step: 3000	Loss : 0.025603
Train Step: 4000	Loss : 0.000938
Train Step: 5000	Loss : 0.000677
Train Step: 6000	Loss : 0.000083
Train Step: 7000	Loss : 0.001494
Train Step: 8000	Loss : 0.034168
Train Step: 9000	Loss : 0.003104
Train Step: 10000	Loss : 0.000211
Train Step: 11000	Loss : 0.000660
Train Step: 12000	Loss : 0.000028
Train Step: 13000	Loss : 0.005091
Train Step: 14000	Loss : 0.000984
Train Step: 15000	Loss : 0.007149
Train Step: 16000	Loss : 0.000481
Train Step: 17000	Loss : 0.000003
Train Step: 18000	Loss : 0.001140
Train Step: 19000	Loss : 0.000851
Train Step: 20000	Loss : 0.000536
Train Step: 21000	Loss : 0.001354
Train Step: 22000	Loss : 0.011733
Train Step: 23000	Loss : 0.071117
Train Step: 24000	Loss : 0.000008
Train Step: 25000	Loss : 0.000003
Train Step: 26000	Loss : 0.000023
Train Step: 27000	Loss : 0.000076
Train Step: 28000	Loss : 0.032359


In [ ]:
# Test
model.eval()
correct = 0
for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()
print('Test set Accuracy :  {:.2f}%'.format(100. *correct / len(test_loader.dataset)))

Test set Accuracy :  99.21%


In [ ]:
# Model Save
PATH = '/content/drive/MyDrive/Colab Notebooks/CNN.pt'

torch.save(model.state_dict(), PATH)